 

# Maximum Area of Histogram

## 1. IP–OP–PS (Input, Output, Problem Statement)

### **Problem Statement**

We are given an array of non-negative integers representing the **heights** of bars in a histogram. Each bar has a **width** of 1. We need to find the **largest rectangular area** that can fit entirely under the histogram.

### **Input**

- An integer \(n\), the number of bars in the histogram.
- An array of \(n\) non-negative integers, e.g. \([6, 2, 5, 4, 5, 1, 6]\).

### **Output**

- A single integer representing the **maximum rectangular area** possible under the histogram.

### **Detailed Example**

**Histogram Heights**:  
\[
[6,\, 2,\, 5,\, 4,\, 5,\, 1,\, 6]
\]

1. One possible rectangle is just the bar at index 0 (height = 6, width = 1) → area = \(6 \times 1 = 6\).  
2. Another rectangle might span indices \([2, 3, 4]\) with height = 4 (the smallest among 5, 4, 5) → width = 3 → area = \(4 \times 3 = 12\).  
3. We need to check all possible rectangles to find the one with the maximum area.  

For this array, the **largest rectangle** has an area of **12** (from the bars with heights \([5, 4, 5]\) where the limiting height is 4 and the width is 3).

---

## 2. Identification

### **Why Use a Stack Technique?**

1. **Naive Approach (O(\(n^2\))):**  
   - We could try every possible pair of bars and compute the minimum height within that range, then multiply by the width. This leads to nested loops and O(\(n^2\)) complexity.

2. **Monotonic Stack Optimization (O(\(n\))):**  
   - By finding the **Nearest Smaller to Left (NSL)** and **Nearest Smaller to Right (NSR)** for each bar, we can determine how far each bar can “expand” without encountering a smaller bar that would reduce the rectangle’s height.  
   - Storing indices in a *monotonically increasing stack* allows each element to be pushed and popped at most once, giving us an O(\(n\)) solution.

3. **Key Cues / Characteristics**:
   - We are effectively looking for how far each bar can extend to the left and right before a bar of **smaller** height is encountered.
   - This is precisely the “nearest smaller element” pattern.

---

## 3. Break Down → Stack Approach

### **Step-by-Step Sub-Tasks**

1. **Compute Nearest Smaller to Left (NSL) indices for each bar**  
   - For bar at index \(i\), find the index of the bar to the left which is **smaller**. If none exists, use a **pseudo-index** \(-1\).  
   - Store these indices in a `left` array.

2. **Compute Nearest Smaller to Right (NSR) indices for each bar**  
   - For bar at index \(i\), find the index of the bar to the right which is **smaller**. If none exists, use a **pseudo-index** \(n\).  
   - Store these indices in a `right` array.

3. **Calculate the width for each bar**  
   - The bar at index \(i\) can extend from `(left[i] + 1)` to `(right[i] - 1)`.  
   - Hence, **width** = `right[i] - left[i] - 1`.

4. **Compute the area for each bar**  
   - **area[i]** = `height[i] * width[i]`.  
   - Keep track of the maximum area encountered.

### **Data Structures Used**

- **Two Stacks (or repeated usage of one stack)**:
  - One stack to compute NSL (traversing from left to right).
  - Another (or re-initialized) stack to compute NSR (traversing from right to left).
- **Arrays**:
  - `left[]` to store the nearest smaller to left indices.
  - `right[]` to store the nearest smaller to right indices.
  - `width[]` and an integer for max area calculation.

---

## 4. Explanations + Code

### **Detailed Explanation of Each Step**

1. **Nearest Smaller to Left (NSL)**  
   - Traverse the histogram from **left to right**.  
   - Maintain a **monotonically increasing** stack of bar indices.  
   - For the current bar \(i\), pop from the stack while the top bar is **greater or equal** in height.  
   - If the stack becomes empty, `left[i] = -1`; otherwise, `left[i] = stack.top()`.  
   - Push the current index \(i\).

2. **Nearest Smaller to Right (NSR)**  
   - Traverse the histogram from **right to left**.  
   - Maintain the same **monotonically increasing** principle (pop while top bar is **greater or equal**).  
   - If empty, `right[i] = n`; else `right[i] = stack.top()`.  
   - Push the current index \(i\).

3. **Calculate Width and Area**  
   - For each bar \(i\),  
     \[
       \text{width}[i] = (\text{right}[i]) - (\text{left}[i]) - 1
     \]  
   - \(\text{area}[i] = \text{height}[i] \times \text{width}[i]\).  
   - Track `max_area = max(max_area, area[i])`.

4. **Time Complexity**  
   - Each bar is pushed and popped at most once in both NSL and NSR passes.  
   - Hence, overall time complexity is **O(\(n\))**.

### **C++ Implementation**

```cpp
#include <iostream>
#include <vector>
#include <stack>
using namespace std;

// Helper function to find Nearest Smaller to Left indices
vector<int> nearestSmallerToLeft(const vector<int> &heights) {
    int n = heights.size();
    vector<int> left(n);
    stack<int> st; // Will store indices

    for(int i = 0; i < n; i++) {
        while(!st.empty() && heights[st.top()] >= heights[i]) {
            st.pop();
        }
        if(st.empty()) {
            left[i] = -1;
        } else {
            left[i] = st.top();
        }
        st.push(i);
    }
    return left;
}

// Helper function to find Nearest Smaller to Right indices
vector<int> nearestSmallerToRight(const vector<int> &heights) {
    int n = heights.size();
    vector<int> right(n);
    stack<int> st;

    for(int i = n - 1; i >= 0; i--) {
        while(!st.empty() && heights[st.top()] >= heights[i]) {
            st.pop();
        }
        if(st.empty()) {
            right[i] = n;
        } else {
            right[i] = st.top();
        }
        st.push(i);
    }
    return right;
}

// Main function to compute maximum area in histogram
int maxAreaHistogram(const vector<int> &heights) {
    int n = heights.size();
    if(n == 0) return 0;

    // Step 1 & 2: Find NSL and NSR
    vector<int> left = nearestSmallerToLeft(heights);
    vector<int> right = nearestSmallerToRight(heights);

    // Step 3: Calculate widths and areas
    int maxArea = 0;
    for(int i = 0; i < n; i++) {
        int width = right[i] - left[i] - 1; // distance between smaller bars
        int area = heights[i] * width;
        maxArea = max(maxArea, area);
    }
    return maxArea;
}

int main() {
    vector<int> histogram = {6, 2, 5, 4, 5, 1, 6};
    int maxArea = maxAreaHistogram(histogram);
    cout << "Maximum Area in Histogram: " << maxArea << endl;
    return 0;
}
```

---

## 5. Animated Visualization

Below is a **Python** snippet using `matplotlib` and `ipywidgets` to visualize how we compute the “Nearest Smaller to Left” and “Nearest Smaller to Right” for each bar, and then calculate the final area. Place this in a Jupyter Notebook cell and use the slider to step through the algorithm.

```python
import matplotlib.pyplot as plt
from ipywidgets import interact, IntSlider

def mah_visual(step_index):
    heights = [6, 2, 5, 4, 5, 1, 6]
    n = len(heights)
    
    # We'll store nearest smaller to left (NSL) and right (NSR) as we go,
    # but for simplicity, let's just illustrate the final area calculation step-by-step.
    
    # Precompute NSL
    left = [0]*n
    stack_l = []
    for i in range(n):
        while stack_l and heights[stack_l[-1]] >= heights[i]:
            stack_l.pop()
        left[i] = stack_l[-1] if stack_l else -1
        stack_l.append(i)
    
    # Precompute NSR
    right = [0]*n
    stack_r = []
    for i in range(n-1, -1, -1):
        while stack_r and heights[stack_r[-1]] >= heights[i]:
            stack_r.pop()
        right[i] = stack_r[-1] if stack_r else n
        stack_r.append(i)
    
    # We'll simulate the final area calculation up to step_index
    # step_index will correspond to bar by bar calculation
    if step_index >= n:
        step_index = n - 1
    
    fig, ax = plt.subplots(figsize=(8,4))
    ax.bar(range(n), heights, color='skyblue')
    ax.set_title(f"Step {step_index + 1}: Calculating area for bar {step_index}")
    ax.set_xlabel("Bar Index")
    ax.set_ylabel("Height")
    
    # Highlight the current bar in orange
    ax.bar(step_index, heights[step_index], color='orange')
    
    # Calculate width and area for the current bar
    current_left = left[step_index]
    current_right = right[step_index]
    width = current_right - current_left - 1
    area = heights[step_index] * width
    
    annotation_text = (f"Nearest Smaller to Left Index: {current_left}\n"
                       f"Nearest Smaller to Right Index: {current_right}\n"
                       f"Width = {current_right} - {current_left} - 1 = {width}\n"
                       f"Area = heights[{step_index}] * width = {heights[step_index]} * {width} = {area}")
    
    ax.text(0.02, 0.95, annotation_text, transform=ax.transAxes,
            fontsize=10, bbox=dict(facecolor='white', alpha=0.7),
            verticalalignment='top')
    
    # Optionally, show partial areas for bars < step_index if you want a full step simulation
    # For brevity, we just highlight the current bar's data.
    
    plt.ylim(0, max(heights)+5)
    plt.show()

interact(mah_visual, step_index=IntSlider(min=0, max=6, step=1, value=0));
```

### **How to Use This Visualization**

1. Copy the code into a Jupyter Notebook cell.  
2. Run the cell.  
3. Adjust the **slider** to select each bar index in turn.  
4. Observe the **Nearest Smaller to Left Index**, **Nearest Smaller to Right Index**, the **calculated width**, and the **area** for that bar.

---

# Summary

1. **Problem Statement (IP–OP–PS):**  
   - We want the largest rectangle that can fit under a histogram defined by an array of bar heights.

2. **Identification:**  
   - A naive approach would be O(\(n^2\)). Using a stack to find Nearest Smaller to Left and Right yields an O(\(n\)) solution.

3. **Breakdown (Stack):**  
   - Compute `left[]` (NSL indices) and `right[]` (NSR indices).  
   - The width for each bar is `(right[i] - left[i] - 1)`.  
   - Multiply width by `heights[i]` to get the area, track the maximum.

4. **Explanations + Code:**  
   - A detailed C++ implementation is provided, with time complexity O(\(n\)).

5. **Animated Visualization:**  
   - The Python snippet (matplotlib + ipywidgets) helps visualize how the stack-based approach identifies the nearest smaller bars and calculates area step-by-step.
 